In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, average_precision_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Import necessary library to run code 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # CPU as Default, GPU= cuda

In [ ]:
# GPU in device available or not will be checked here. GPU is needed for heavy programming.

In [ ]:
#Hyper parameter                     # Related to accuracy
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

In [ ]:
# Hyper parameter includes batch_size, num_classes, learning_rate, num_epochs which is related to accuracy

**Load the MNIST Data**

In [ ]:
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])

In [ ]:
# resize data into 32*32 dimension ,preserved it into transform

In [ ]:
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transform,
                                           download = True)



In [ ]:
# Data is saven into ./data folder in our PC. For train dataset train will be "True".

In [ ]:
test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transform,
                                          download=True)

In [ ]:
# Data is saven into ./data folder in our PC. For test dataset train will be "False".

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,                   # devide dataset according to batch size
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [ ]:
for X, y in train_loader:                                     # chech the train data dimension
  print(X.shape)
  break

torch.Size([64, 1, 32, 32])


**LeNet-5 Model**

In [ ]:
                                                                           #convolution layer - Feature Extract , FC Layer - perform regression/classification
                                                                           # For multiclass we use Softmax Activation function

class LeNet(nn.Module):                                                 
    def __init__(self):                                                  
        super(LeNet, self).__init__()
        self.conv_layers= nn.Sequential(
            nn.Conv2d(1,6,5), #[1, 6, 28, 28] #Gray image                 
            nn.ReLU(),
            nn.AvgPool2d(2,2), #[1, 6, 14, 14]
            nn.Conv2d(6,16,5), #[1, 16, 10, 10]
            nn.ReLU(),
            nn.AvgPool2d(2,2), #[1, 16, 5, 5]
        )
        self.fc_layers= nn.Sequential(
            nn.Linear(16*5*5, 120), #120 neurons
            nn.ReLU(),
            nn.Linear(120,84), #84 neurons
            nn.ReLU(),
            nn.Linear(84,num_classes), #10 neurons

        )

    def forward(self, x):
        x= self.conv_layers(x)
        x= x.reshape(x.shape[0], -1)
        x= self.fc_layers(x)
        return x

In [ ]:
# Firstly here used convolution layer for extract data. Then  Fully connected layer used to Classification task which needed.

In [ ]:
model = LeNet()
model.to(device)

LeNet(
  (conv_layers): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

**Loss and optimizer**

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)                        # Adam optimizer is used to update the model's parameters 

**Train the model**

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.2153
Epoch [1/10], Step [800/938], Loss: 0.1203
Epoch [2/10], Step [400/938], Loss: 0.3141
Epoch [2/10], Step [800/938], Loss: 0.0695
Epoch [3/10], Step [400/938], Loss: 0.1782
Epoch [3/10], Step [800/938], Loss: 0.0137
Epoch [4/10], Step [400/938], Loss: 0.0914
Epoch [4/10], Step [800/938], Loss: 0.0073
Epoch [5/10], Step [400/938], Loss: 0.0450
Epoch [5/10], Step [800/938], Loss: 0.0124
Epoch [6/10], Step [400/938], Loss: 0.0145
Epoch [6/10], Step [800/938], Loss: 0.0136
Epoch [7/10], Step [400/938], Loss: 0.0328
Epoch [7/10], Step [800/938], Loss: 0.0157
Epoch [8/10], Step [400/938], Loss: 0.1374
Epoch [8/10], Step [800/938], Loss: 0.0043
Epoch [9/10], Step [400/938], Loss: 0.0063
Epoch [9/10], Step [800/938], Loss: 0.0223
Epoch [10/10], Step [400/938], Loss: 0.0004
Epoch [10/10], Step [800/938], Loss: 0.0016


In [ ]:
# train the model over multiple epochs, iterating through the training data in batches.
# In each iteration, calculate the loss, perform backpropagation, and update the model's weights using the optimizer.

**Accuracy**

In [ ]:
true_labels = []
predicted_labels = []

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)


        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy= 100 * correct / total
    print('Accuracy of the network on the 10000 test images: {} %'.format(accuracy))
    top1_error = 100 - accuracy
    print('Top-1 Error: {} %'.format(top1_error))



Accuracy of the network on the 10000 test images: 98.73 %
Top-1 Error: 1.269999999999996 %


In [ ]:
# After training, evaluate the model's accuracy on the test dataset.
# calculated the top-1 error as well.

**Confusion matrix**

In [ ]:
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
confusion = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(num_classes, num_classes))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', square=True, xticklabels=True, yticklabels=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# create a confusion matrix to visualize the model's performance in classifying each digit.
# seaborn is used to create a heatmap for the confusion matrix.

**Classification report**

In [ ]:
report = classification_report(true_labels, predicted_labels)
print('\n'+ report)


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.97      1.00      0.98      1010
           4       0.99      0.99      0.99       982
           5       1.00      0.97      0.98       892
           6       0.99      0.99      0.99       958
           7       1.00      0.98      0.99      1028
           8       0.98      0.99      0.98       974
           9       0.97      0.99      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [ ]:
# generate a classification report, including metrics like precision, recall, and F1-score, for each class (0-9).

In [ ]:
# Calculate precision, recall, f1 score
precision = precision_score(true_labels, predicted_labels, average=None)
recall = recall_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)


for class_label in range(10):
    print(f'Class {class_label}:')
    print(f'Precision: {precision[class_label]:.2f}')
    print(f'Recall: {recall[class_label]:.2f}')
    print(f'F1-score: {f1[class_label]:.2f}')
    print()


Class 0:
Precision: 0.99
Recall: 0.99
F1-score: 0.99

Class 1:
Precision: 0.99
Recall: 1.00
F1-score: 0.99

Class 2:
Precision: 0.99
Recall: 0.98
F1-score: 0.98

Class 3:
Precision: 0.97
Recall: 1.00
F1-score: 0.98

Class 4:
Precision: 0.99
Recall: 0.99
F1-score: 0.99

Class 5:
Precision: 1.00
Recall: 0.97
F1-score: 0.98

Class 6:
Precision: 0.99
Recall: 0.99
F1-score: 0.99

Class 7:
Precision: 1.00
Recall: 0.98
F1-score: 0.99

Class 8:
Precision: 0.98
Recall: 0.99
F1-score: 0.98

Class 9:
Precision: 0.97
Recall: 0.99
F1-score: 0.98



**Top1 error**

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)* 100
print(f'Accuracy: {accuracy:.2f}%')
top1_error = 100 - accuracy
print('Top-1 Error: {} %'.format(top1_error))

Accuracy: 98.73%
Top-1 Error: 1.2700000000000102 %


In [ ]:
# calculate the top-1 error, which is the complement of accuracy.